# Problem

In [ ]:

Şirketi terk edecek müşterileri tahmin edebilecek bir makine öğrenmesi modeli geliştirebilir misiniz?

Amaç bir bankanın müşterilerinin bankayı terk etme ya da terk etmeme durumunun tahmin edilmesidir.

Müşteri terkini tanımlayan olay müşterinin banka hesabını kapatmasıdır.

Veri Seti Hikayesi:

10000 gözlemden ve 12 değişkenden oluşmaktadır.
Bağımsız değişkenler müşterilere ilişkin bilgiler barındırmaktadır.
Bağımlı değişken müşteri terk durumunu ifade etmektedir.
Değişkenler:

Surname : Soy isim
CreditScore : Kredi skoru
Geography : Ülke (Germany/France/Spain)
Gender : Cinsiyet (Female/Male)
Age : Yaş
Tenure : Kaç yıllık müşteri
Balance : Bakiye
NumOfProducts : Kullanılan banka ürünü
HasCrCard : Kredi kartı durumu (0=No,1=Yes)
IsActiveMember : Aktif üyelik durumu (0=No,1=Yes)
EstimatedSalary : Tahmini maaş
Exited : Terk mi değil mi? (0=No,1=Yes)

In [ ]:


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Understanding

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression  
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, GridSearchCV,train_test_split,cross_val_score
import itertools
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import LocalOutlierFactor # çok değişkenli aykırı gözlem incelemesi
from sklearn.preprocessing import scale,StandardScaler, MinMaxScaler,Normalizer,RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import  accuracy_score, f1_score, precision_score,confusion_matrix, recall_score, roc_auc_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

%config InlineBackend.figure_format = 'retina'

# to display all columns and rows:
pd.set_option('display.max_columns', None); pd.set_option('display.max_rows', None);
df = pd.read_csv("../input/churn-for-bank-customers/churn.csv" , index_col=0)

# 1.Data Preprocessing

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df["Exited"].value_counts()*100/len(df)


In [ ]:


labels = 'Exited', 'Retained'
sizes = [df.Exited[df['Exited']==1].count(), df.Exited[df['Exited']==0].count()]
explode = (0, 0.1)
fig1, ax1 = plt.subplots(figsize=(10, 8))
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.title("Proportion of customer churned and retained", size = 20)
plt.show()

In [ ]:
fig, axarr = plt.subplots(2, 2, figsize=(20, 12))
sns.countplot(x='Geography', hue = 'Exited',data = df, ax=axarr[0][0])
sns.countplot(x='Gender', hue = 'Exited',data = df, ax=axarr[0][1])
sns.countplot(x='HasCrCard', hue = 'Exited',data = df, ax=axarr[1][0])
sns.countplot(x='IsActiveMember', hue = 'Exited',data = df, ax=axarr[1][1])

In [ ]:
df = df.drop([ "CustomerId", "Surname" ], axis = 1)

In [ ]:
df.head()

In [ ]:
import seaborn as sns
sns.pairplot(df, hue = "Exited");

In [ ]:
# burada bağımlı değişkeni, "Exited" u yukarıdaki gibi satıra ve sütüna koymadan gösterdik. yani "Exited" gösterip boş göstermedi
# regresyon eğrilerin icin kind argumanini kullaniyorum 
# burada sdf dataframindeki ["Age", "Balance","Tenure","EstimatedSalary","CreditScore"] değişkenleri seçtik.
sns.pairplot(df, vars = ["Age", "Balance","Tenure","EstimatedSalary","CreditScore"], 
             hue = "Exited",
            kind = "reg");

# 1.1 one hot encoding

In [ ]:
df = pd.get_dummies(df, columns = {"Gender", "Geography"})


In [ ]:
df.head()

# 1.2 outliers

In [ ]:
for feature in df:
    
    Q1 = df[feature].quantile(0.05)
    Q3 = df[feature].quantile(0.95)
    IQR = Q3-Q1
    lower = Q1- 1.5*IQR
    upper = Q3 + 1.5*IQR
    
    if df[(df[feature] > upper)].any(axis=None):
        print(feature,"yes")
    else:
        print(feature, "no")

In [ ]:
sns.boxplot(x=df.NumOfProducts);

In [ ]:
Q1 = df["NumOfProducts"].quantile(0.05)
Q3 = df["NumOfProducts"].quantile(0.95)
IQR = Q3-Q1
upper = Q3 + 1.5*IQR
upper

In [ ]:
df[df["NumOfProducts"] > upper]
df.loc[df["NumOfProducts"] > upper, "NumOfProducts"] = upper

In [ ]:
df.groupby("Exited" ).agg({"CreditScore":"mean" , "Tenure":"mean" ,"Balance":"mean",  } ).head()

# 2. FEATURE ENGINEERING

ÖN BILGI :

Kredi ya da kredi kartı vermek için bankanın belirlediği alt sınır bankadan bankaya, almak istediğiniz tutara ve gelir durumunuza göre değişse bile Findeks kredi notu aralıklarının anlamı şöyle:

Kredi Notu

0 - 699  = Risk Grubu

700 - 1099 = En riskli

1100 - 1499 = Orta riskli

1500 - 1699 = Az riskli

1700 - 1900 = Iyi

1500 - 1900 = Çok İyi

In [ ]:
df["CreditScore"].max()

In [ ]:
df["CreditScore"].min()

In [ ]:
# sonuc olarak verimizde max ve min kredi skoru 350 ve 850 arsinda degisiyor

In [ ]:
def CreditScore(kredi_skoru):
    
    if 300<= kredi_skoru < 500:
        return "Cok riskli"
    elif 500<=kredi_skoru < 700:
        return "Riskli"
    elif  700<= kredi_skoru < 800:
        return "Iyi"
    elif  800<=  kredi_skoru < 850:
        return "Cok iyi"
    elif  850 <= kredi_skoru: 
        return "Harika"
    elif 300 > kredi_skoru :
        return "Berbat"
    
df['New_CreditScore_status'] = df['CreditScore'].apply(CreditScore)
    


In [ ]:
# #bakiye maas orani

In [ ]:
df['New_BalanceSalaryRate'] = df.Balance/df.EstimatedSalary


In [ ]:
df.head()

In [ ]:
#OneHotEncoding yeni degisken icin yapmam lazim

In [ ]:
df =pd.get_dummies(df, columns = ["New_CreditScore_status"], drop_first = True, dummy_na = True)

In [ ]:
df.columns


In [ ]:
df.head(20)

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.New_CreditScore_status_nan.head()

In [ ]:
df = df.drop([ "New_CreditScore_status_nan" ], axis = 1)

In [ ]:
y = df["Exited"]
X = df.drop(["Exited"], axis = 1)

# 3.Standardization

In [ ]:
scaled_cols=StandardScaler().fit_transform(X)
scaled_cols=pd.DataFrame(scaled_cols, columns=X.columns)
scaled_cols.head()

# 4.Modeling

In [ ]:
# Tum modellerin train validasyon skorları

models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('SVM', SVC(gamma='auto')))
models.append(('XGB', GradientBoostingClassifier()))

# evaluate each model in turn
results = []
names = []


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 12345)


In [ ]:


results = []
names = []

for name, model in models:
    
        kfold = KFold(n_splits = 10, random_state = 12345)
        cv_results = cross_val_score(model, X_train, y_train, cv = kfold, scoring= "accuracy")
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)


In [ ]:
for name, model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        msg = "%s: (%f)" % (name, acc)
        print(msg)

# 4.1 Model Tuning

# 4.1.1 Model Installation

#  RandomForest

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV

In [ ]:
rf_model = RandomForestClassifier(random_state = 12345).fit(X_train, y_train)

In [ ]:
y_pred = rf_model.predict(X_test)

In [ ]:
print("cross_val_score : ", cross_val_score(rf_model, X_train, y_train, cv = 10).mean())

In [ ]:
# RandomForest Tuning

In [ ]:
rf_params = {"max_depth": [5,8,10,None],
            "max_features": [2,5,10,15,17],
            "n_estimators": [100,200, 500, 1000],
            "min_samples_split": [2,5,10,20,30]}

In [ ]:
rf_cv_model = RandomizedSearchCV(estimator= rf_model, param_distributions = rf_params, n_iter=50, cv=10, scoring='accuracy', n_jobs=-1, verbose=2).fit(X, y)

In [ ]:
rf_cv_model.best_params_

In [ ]:
rf_tuned = RandomForestClassifier(**rf_cv_model.best_params_).fit(X_train, y_train)

In [ ]:
y_pred = rf_tuned.predict(X_test)

print("cross_val_score : ", cross_val_score(rf_tuned, X_train, y_train, scoring='accuracy', cv = 10, n_jobs=-1, verbose=2).mean())

In [ ]:
from sklearn.metrics import accuracy_score
print('RandomForest model accuracy score: {0:0.2f}'. format(accuracy_score(y_test, y_pred)))

# XGB

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV


In [ ]:
xgb = XGBClassifier().fit(X_train,y_train)
y_pred = xgb.predict(X_test)

print("cross_val_score : ", cross_val_score(xgb, X_train, y_train, cv = 10).mean())


In [ ]:
# XGB Tuning 

In [ ]:
xgb_params = {"learning_rate": [0.1,0.01,0.5],
             "max_depth": [2,3,4,5,8],
             "n_estimators": [100,200,500,1000],
             "colsample_bytree": [0.4,0.7,1]}

In [ ]:
xgb_rsc = RandomizedSearchCV(estimator=xgb, param_distributions=xgb_params, n_iter=50, cv=10, scoring='accuracy', n_jobs=-1, verbose=2).fit(X,y)

In [ ]:
xgb_rsc.best_params_

In [ ]:
xgb_tuned = XGBClassifier(**xgb_rsc.best_params_).fit(X_train, y_train)

In [ ]:
y_pred = xgb_tuned.predict(X_test)

In [ ]:
print("cross_val_score : ", cross_val_score(xgb_tuned, X_train, y_train, cv = 10).mean())

In [ ]:
from sklearn.metrics import accuracy_score
print('XGBoost model accuracy score: {0:0.2f}'. format(accuracy_score(y_test, y_pred)))

# 5. FEATURE IMPORTANCE

# 5.1   XGB

In [ ]:

Importance = pd.DataFrame({'Importance':xgb_tuned.feature_importances_*100},
                          index = X_train.columns)
Importance.sort_values(by = 'Importance',
                       axis = 0,
                       ascending = True).plot(kind = 'barh',
                                              color = 'r', )
plt.xlabel('Variable Importance')
plt.gca().legend_ = None

# 5.2 RandomForest

In [ ]:
Importance = pd.DataFrame({'Importance':rf_tuned.feature_importances_*100},
                          index = X_train.columns)
Importance.sort_values(by = 'Importance',
                       axis = 0,
                       ascending = True).plot(kind = 'barh',
                                              color = 'r', )
plt.xlabel('Variable Importance')
plt.gca().legend_ = None

# 6.Confusion Matrix

In [ ]:
from matplotlib import rc,rcParams
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.rcParams.update({'font.size': 19})
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title,fontdict={'size':'16'})
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45,fontsize=13,color="blue")
    plt.yticks(tick_marks, classes,fontsize=12,color="blue")
    rc('font', weight='bold')
    fmt = '.1f'
    thresh = cm.max()
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="red")

    plt.ylabel('True label',fontdict={'size':'16'})
    plt.xlabel('Predicted label',fontdict={'size':'16'})
    plt.tight_layout()


In [ ]:
cfm = confusion_matrix(y_test, y_pred=y_pred)
plot_confusion_matrix(cfm, classes=['Non Exited','Exited'],
                      title='Churn Confusion matrix')
tn, fp, fn, tp = cfm.ravel()
print("True Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)
print("True Positives: ",tp)

# 7.Reporting

#  Churn veri seti input edildi.

#  1. Veri analizi veri ön isleme yapildi.
#    * Aykiri gözlem yapildi
#    * One hot encoding yapildi

#  2.Feature Enginering yapildi

#  3.Standardization

#  4.Alti  tane  Model ayni anda kuruldu ve en iyi modelllrin RandomForest  ve XGB oldugu saptandi. 

#  4.1.RandomForest ve XGB modeli  tun edildi ve en iyi sonuc amaclandi.

#  5.Degisken onem siralarina bakildi bu iki modelde.

#  6.Model sonucu: sonuc sayilari yazdirilarak görselestirildi.       